In [4]:
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm

import sys

## Path for Dataset
path = ''
sys.path.insert(0, path)

In [2]:
read = h5py.File(path + 'Triggers.hdf5')
write = h5py.File(path + 'Coinc_manual.h5py', 'w')
hwt = write.create_group('Htrig')
lwt = write.create_group('Ltrig')
hws = write.create_group('Hsnr')
lws = write.create_group('Lsnr')
tofw = write.create_group('tof')

In [3]:
tof = 0.0105 + 1/256*2

for k, z in zip(read['H']['trig'].keys(), tqdm(range(236), desc="Loading" )):
    
    HTR = []
    LTR = []
    HS = []
    LS = []
    TOF = []
    
    Htrig = np.array(read['H']['trig'][k])
    Ltrig = np.array(read['L']['trig'][k])

    Hsnr = np.array(read['H']['snr'][k])
    Lsnr = np.array(read['L']['snr'][k])
    index = np.arange(len(Lsnr))
    
    #Htrig = [3.414134, 5.43223144, 7.5632147, 9, 10, 14, 19, 132, 1234, 123544, 100000000000]
    #Ltrig = [0.1, 0.9, 1.5, 2, 4, 7.5632147, 19, 123454322]
    
    for i in range(len(Htrig)):
        
        ht = Htrig[i]
        
        low = 0
        high = len(Ltrig)-1
        keep = True
        
        count = 0
        while keep:
        
                
            count += 1
            mid = int((low + high)/2)
            
            if abs(Ltrig[mid]-ht) <= tof:
                keep = False
            
            elif high <= low:
                keep = False
                mid = -1
                
                if abs(Ltrig[high]-ht) <= tof:
                    mid = high
                    
            elif low == high - 1:
                
                keep = False
                if abs(Ltrig[high]-ht) <= tof:
                    mid = high
                elif abs(Ltrig[low]-ht) <= tof:
                    mid = low
                else:
                    mid = -1
            
            else:
                if abs(Ltrig[mid]-ht) <= tof:
                    keep = False
            
                elif ht < Ltrig[mid]:
                    high = mid
                else:
                    low = mid
                    
        
        if mid != -1:
            
            HTR.append(Htrig[i])
            LTR.append(Ltrig[mid])
            
            HS.append(Hsnr[i])
            LS.append(Lsnr[mid])
            
            TOF.append(abs(Htrig[i] - Ltrig[mid]))
            
            
            
            
            
    hwt.create_dataset(k, data=HTR)
    lwt.create_dataset(k, data=LTR)
    
    hws.create_dataset(k, data=HS)
    lws.create_dataset(k, data=LS)
    tofw.create_dataset(k, data=TOF)
        
            
write.close()

Loading: 100%|███████████████████████████████▊| 235/236 [02:14<00:00,  1.75it/s]
